In [16]:
import pandas as pd
import json

In [3]:
df = pd.read_csv("data/AKG/Test Collection/AKGG/akg_standard_akgg_property_rele.csv")

In [13]:
df.property.nunique()

296

In [35]:
import json

with open("data/AKG/Formal Run Topics/AKGG_Formal_Run_Topic.json") as json_file:
    data = json.load(json_file)
    qid, query, entity, entityType, action = [], [], [], [], []
    for p in data['queries']:
        qid.append(p['queryId'])
        query.append(p['query'])   
        entity.append(p['entity'])
        entityType.append(' '.join(p['entityTypes']))    
        action.append(p['action'])
topic = pd.DataFrame({"query_id": qid, "query": query, "entity": entity, "entityType": entityType, "action":action})

In [99]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

In [156]:

with open("data/AKG/Participants Runs/AKGG/akgg-formalrun-cuis.json") as json_file:
    data = json.load(json_file)
    run = AutoVivification()
    for p in data['runs']:
        for res in p['results']:
            for prop in res['properties']:
                run[p['runid']][str(res['queryid'])][str(prop['property'])] = prop['rank']


In [31]:
df = df.merge(topic, how="inner", on="query_id")

In [37]:
df.entityType.nunique()

41

In [39]:
df.entity.nunique()

229

In [55]:
import collections
qrel = collections.defaultdict(dict)
for qid, prop, label in df[['query_id', 'property', 'rele_label']].values:
    qrel[str(qid)][str(prop)] = int(label)

In [204]:
import pytrec_eval
import json
import numpy as np
def read_results(results):
    map = np.mean([i['map'] for i in results.values()])
    ndcg = np.mean([i['ndcg_cut_10'] for i in results.values()])
    mrr = np.mean([i['recip_rank'] for i in results.values()])
    return map, ndcg, mrr

evaluator = pytrec_eval.RelevanceEvaluator(
    tmp, {'map', 'ndcg_cut', 'recip_rank'})

results = evaluator.evaluate(run['1'])

map, ndcg, mrr = read_results(results)

print(map, ndcg, mrr)

0.4176866906743068 0.35882559429220734 0.5


In [170]:
def read_results(results):
    map = np.mean([i['map'] for i in results.values()])
    ndcg = np.mean([i['ndcg'] for i in results.values()])
    mrr = np.mean([i['recip_rank'] for i in results.values()])
    return map, ndcg, mrr

def evaluate(evaluator, df, pred):
    # map prediction to trec format
    pred = np.array(pred)
    run = collections.defaultdict(dict)
    for qid, p, label in zip(df['id'], df['pid'], pred):
        run[str(qid)][str(p)] = float(label)
    results = evaluator.evaluate(run)

    map, ndcg, mrr = read_results(results)

    return results, map, ndcg, mrr

In [281]:
from pyNTCIREVAL import Labeler
from pyNTCIREVAL.metrics import MSnDCG, nERR

res = []
for i in qrel:
    # dict of { document ID: relevance level }
    qrels = qrel[str(i)] 
    grades = [1,2,3,4] # a grade for relevance levels 1 and 2 (Note that level 0 is excluded)
#     grades = list(set(qrels.values())) + [0]
    ranked_list = run['1'][str(i)] # a list of document IDs

    # labeling: [doc_id] -> [(doc_id, rel_level)]
    labeler = Labeler(qrels)
    labeled_ranked_list = labeler.label(ranked_list)


    # compute the number of documents for each relevance level
    rel_level_num = 5
    xrelnum = labeler.compute_per_level_doc_num(rel_level_num)

    # Let's compute nDCG@5
    metric = MSnDCG(xrelnum, grades, cutoff=10)
    result = metric.compute(labeled_ranked_list)
    res.append(result)

print(np.mean(res))

0.5324041751716548


In [282]:
qrel

defaultdict(dict,
            {'1': {'startTime': 2,
              'result': 3,
              'target': 1,
              'name': 3,
              'actionStatus': 3,
              'object': 3,
              'instrument': 1,
              'disambiguatingDescription': 2,
              'description': 2,
              'participant': 2,
              'agent': 3},
             '2': {'potentialAction': 3,
              'participant': 3,
              'startTime': 3,
              'endTime': 2,
              'instrument': 2,
              'result': 2,
              'additionalType': 2,
              'image': 2,
              'object': 2},
             '3': {'result': 3,
              'image': 2,
              'participant': 2,
              'description': 1,
              'potentialAction': 2,
              'location': 2,
              'startTime': 1,
              'target': 1,
              'endTime': 1,
              'alternateName': 1,
              'additionalType': 4},
             '4': {'

In [248]:
len(qrel), len(run['1'])

(229, 229)

In [229]:
qrel

defaultdict(dict,
            {'1': {'startTime': 2,
              'result': 3,
              'target': 1,
              'name': 3,
              'actionStatus': 3,
              'object': 3,
              'instrument': 1,
              'disambiguatingDescription': 2,
              'description': 2,
              'participant': 2,
              'agent': 3},
             '2': {'potentialAction': 3,
              'participant': 3,
              'startTime': 3,
              'endTime': 2,
              'instrument': 2,
              'result': 2,
              'additionalType': 2,
              'image': 2,
              'object': 2},
             '3': {'result': 3,
              'image': 2,
              'participant': 2,
              'description': 1,
              'potentialAction': 2,
              'location': 2,
              'startTime': 1,
              'target': 1,
              'endTime': 1,
              'alternateName': 1,
              'additionalType': 4},
             '4': {'

In [20]:

def read_query_json():
    queries = []
    with open("data/AKG/Formal Run Topics/AKGG_Formal_Run_Topic.json", encoding="utf-8") as file:
        for l in file:
            y = json.loads(l)
            queries.append(y['queries'])
    print(queries)

df_video = read_query_json()

[[{'queryId': 1, 'query': 'benchmarking', 'entity': 'Benchmarking', 'entityTypes': ['Thing', 'Action'], 'action': 'have'}, {'queryId': 2, 'query': 'bodybuilding', 'entity': 'Bodybuilding', 'entityTypes': ['Thing', 'Action'], 'action': 'take'}, {'queryId': 3, 'query': 'brainstorming', 'entity': 'Brainstorming', 'entityTypes': ['Thing', 'Action'], 'action': 'brainstorm'}, {'queryId': 4, 'query': 'chain smoking', 'entity': 'Chain smoking', 'entityTypes': ['Thing', 'Action'], 'action': 'start'}, {'queryId': 5, 'query': 'funding process', 'entity': 'Funding', 'entityTypes': ['Thing', 'Action'], 'action': 'get funding'}, {'queryId': 6, 'query': 'hair coloring process', 'entity': 'Hair coloring', 'entityTypes': ['Thing', 'Action'], 'action': 'bleach hair back to brown'}, {'queryId': 7, 'query': 'hit and run', 'entity': 'Hit and run', 'entityTypes': ['Thing', 'Action'], 'action': 'stop at the scene of an accident'}, {'queryId': 8, 'query': 'Language acquisition', 'entity': 'Language acquisitio